# Find KEGG associations

This notebokk will create a table that has the KEGG pathways that are associated with all genes, but we are particularly interested in those that are associated with the most and least stable genes.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import random
import pandas as pd
from scripts import paths, utils, modules, annotations

random.seed(1)

In [2]:
# Output files
pao1_out_filename = "pao1_core_similarity_associations.tsv"
pa14_out_filename = "pa14_core_similarity_associations.tsv"

In [3]:
# Load transcriptional similarity df
pao1_similarity_scores_filename = "pao1_similarity_scores.tsv"
pa14_similarity_scores_filename = "pa14_similarity_scores.tsv"

pao1_similarity_scores = pd.read_csv(
    pao1_similarity_scores_filename, sep="\t", header=0, index_col=0
)
pa14_similarity_scores = pd.read_csv(
    pa14_similarity_scores_filename, sep="\t", header=0, index_col=0
)

In [4]:
print(pao1_similarity_scores.shape)
pao1_similarity_scores.head()

(5349, 5)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label
PAO1 id,,,,,
PA1642,PA14_43280,0.393037,4.089216e-197,selD,NaN
PA0481,PA14_06280,0.242300,2.526173e-72,NaN,NaN
PA3714,PA14_16350,0.335471,7.698982e-141,NaN,NaN
PA1164,PA14_49350,0.389940,8.702679e-194,NaN,NaN
PA0749,PA14_54600,0.336135,2.004630e-141,NaN,NaN


In [5]:
print(pa14_similarity_scores.shape)
pa14_similarity_scores.head()

(5355, 5)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label
PA14_00010,PA0001,0.432963,1.738111e-243,dnaA,NaN
PA14_00020,PA0002,0.525193,0.000000e+00,dnaN,most stable
PA14_00030,PA0003,0.359931,1.974611e-163,recF,NaN
PA14_00050,PA0004,0.537333,0.000000e+00,gyrB,most stable
PA14_00060,PA0005,0.335001,1.664864e-140,lptA,NaN


In [6]:
# Load KEGG pathway data
pao1_pathway_filename = "https://raw.githubusercontent.com/greenelab/adage/7a4eda39d360b224268921dc1f2c14b32788ab16/Node_interpretation/pseudomonas_KEGG_terms.txt"

In [7]:
pao1_pathways = annotations.load_format_KEGG(pao1_pathway_filename)
pao1_pathways.head()

,1,2
0,,
KEGG-Pathway-pae00072: Synthesis and degradation of ketone bodies,10,"{PA2000, PA2003, PA2001, PA4785, PA3925, PA358..."
KEGG-Pathway-pae00071: Fatty acid degradation,32,"{PA1535, PA3629, PA3300, PA3925, PA3589, PA174..."
KEGG-Pathway-pae00903: Limonene and pinene degradation,9,"{PA1027, PA1821, PA4899, PA2475, PA3331, PA173..."
KEGG-Pathway-pae00380: Tryptophan metabolism,27,"{PA4613, PA3925, PA4236, PA3589, PA0421, PA174..."
KEGG-Pathway-pae00900: Terpenoid backbone biosynthesis,16,"{PA2001, PA4785, PA3925, PA3633, PA3589, PA404..."


## Pathway annotations to PA14

The annotations we have are only for PAO1 genes, so we will map PAO1 core genes to PA14 core genes to add annotations to PA14. This is possible since we are focused on only core genes, which have homologs between PAO1 and PA14

In [8]:
pao1_annotation_filename = paths.GENE_PAO1_ANNOT
gene_mapping_pao1 = utils.get_pao1_pa14_gene_map(pao1_annotation_filename, "pao1")

In [9]:
gene_mapping_pao1 = gene_mapping_pao1["PA14_ID"].to_frame()

## Get pathway associations for all genes

In [10]:
def get_associated_pathways(genes_):
    rows = []
    for gene_id in genes_:
        pathway_bool = [
            gene_id in pao1_pathways.loc[pathway, 2] for pathway in pao1_pathways.index
        ]
        found_pathways = list(pao1_pathways[pathway_bool].index)
        rows.append({"gene id": gene_id, "pathways present": found_pathways})
    return pd.DataFrame(rows).set_index("gene id")

In [11]:
# Get KEGG associations for all genes in PAO1
all_pao1_gene_ids = list(pao1_similarity_scores.index)
pao1_associations = get_associated_pathways(all_pao1_gene_ids)

In [12]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 1)


,pathways present
gene id,
PA1642,[KEGG-Pathway-pae00450: Selenocompound metabol...
PA0481,[]
PA3714,[]
PA1164,[]
PA0749,[]


In [13]:
# Map PA14 gene ids
pa14_associations = pao1_associations.merge(
    gene_mapping_pao1, left_index=True, right_index=True
)
pa14_associations.set_index("PA14_ID", inplace=True)
print(pa14_associations.shape)
pa14_associations.head()

(5349, 1)


,pathways present
PA14_ID,
PA14_43280,[KEGG-Pathway-pae00450: Selenocompound metabol...
PA14_06280,[]
PA14_16350,[]
PA14_49350,[]
PA14_54600,[]


In [14]:
# Merge KEGG associations with transcriptional similarity information
pao1_associations = pao1_similarity_scores.merge(
    pao1_associations, left_index=True, right_index=True, how="left"
)
pa14_associations = pa14_similarity_scores.merge(
    pa14_associations, left_index=True, right_index=True, how="left"
)

In [15]:
print(pao1_associations.shape)
pao1_associations.head()

(5349, 6)


,PA14 homolog id,Transcriptional similarity across strains,P-value,Name,label,pathways present
PAO1 id,,,,,,
PA1642,PA14_43280,0.393037,4.089216e-197,selD,NaN,[KEGG-Pathway-pae00450: Selenocompound metabol...
PA0481,PA14_06280,0.242300,2.526173e-72,NaN,NaN,[]
PA3714,PA14_16350,0.335471,7.698982e-141,NaN,NaN,[]
PA1164,PA14_49350,0.389940,8.702679e-194,NaN,NaN,[]
PA0749,PA14_54600,0.336135,2.004630e-141,NaN,NaN,[]


In [16]:
print(pa14_associations.shape)
pa14_associations.head()

(5355, 6)


,PAO1 homolog id,Transcriptional similarity across strains,P-value,Name,label,pathways present
PA14_00010,PA0001,0.432963,1.738111e-243,dnaA,NaN,[]
PA14_00020,PA0002,0.525193,0.000000e+00,dnaN,most stable,[KEGG-Pathway-pae03440: Homologous recombinati...
PA14_00030,PA0003,0.359931,1.974611e-163,recF,NaN,[KEGG-Pathway-pae03440: Homologous recombination]
PA14_00050,PA0004,0.537333,0.000000e+00,gyrB,most stable,[]
PA14_00060,PA0005,0.335001,1.664864e-140,lptA,NaN,[KEGG-Pathway-pae00561: Glycerolipid metabolis...


In [17]:
# Save
pao1_associations.to_csv(pao1_out_filename, sep="\t")
pa14_associations.to_csv(pa14_out_filename, sep="\t")

**Takeaway:**

Based on the pathways associated with the most and least stable core genes, we find that the most stable core genes tend to be associated with pathways related to cellular maintenance including: protein transport systems, ribosomes, metabolism, type III, IV secretion system which mediates virulence.

There are far fewer KEGG pathways that least stable core genes are found to be associated with. The least stable core genes are mostly associated with different types of metabolism.

A google doc containing the most and least stable core genes and some information about them is [here](https://docs.google.com/spreadsheets/d/1SqEyBvutfbsOTo4afg9GiEzP32ZKplkN1a6MpAQBvZI/edit?usp=sharing).